In [102]:
import psycopg2
import os
import requests
import time
import shapely.wkb
import shapely.ops
import sys
import pyproj
from functools import partial

In [74]:
# setup postgis connecton funcitons
pg_user=os.environ.get('POSTGIS_ENV_POSTGRES_USER')
pg_pass=os.environ.get('POSTGIS_ENV_POSTGRES_PASSWORD')
pg_host=os.environ.get('POSTGIS_PORT_5432_TCP_ADDR')
pg_port=os.environ.get('POSTGIS_PORT_5432_TCP_PORT')

def postgis(query):
    conn = psycopg2.connect(user=pg_user, 
                        password=pg_pass,
                        host=pg_host,
                        port=pg_port
                       )
    cur = conn.cursor()
    cur.execute(query)
    r = list(cur.fetchall())
    cur.close()
    conn.close()
    return r

In [33]:
# setup carto connection funcitons
CARTO_URL = 'https://{}.carto.com/api/v2/sql'
CARTO_USER = 'wri-01'
CARTO_KEY = os.environ.get('CARTO_KEY')

def sendSql(sql, user=None, key=None):
    '''Send arbitrary sql and return response object or False'''
    user = user or CARTO_USER
    key = key or CARTO_KEY
    url = CARTO_URL.format(user)
    payload = {
        'api_key': key,
        'q': sql,
    }
    r = requests.post(url, json=payload)
    if (r.status_code >= 400):
        try:
            msg = r.json()['error'][0]
        except:
            r.raise_for_status()
        raise Exception(msg)
    return r.json()

In [ ]:
# get test geoms
gadm_table2 = 'gadm36_adm2'
wdpa_table = 'wdpa_protected_areas'

query = """
select cartodb_id, gid_2 from {} where gid_2 in ('USA.2.13_1', 'USA.2.18_1', 'USA.2.2_1')
""".format(gadm_table2)
ids = [r['cartodb_id'] for r in sendSql(query)['rows']]
tolerance = 0.083333

start = time.perf_counter()

test_geoms = []
for cdbid in ids:
    query = """
        SELECT
            ST_INTERSECTION(a.the_geom, b.the_geom) AS the_geom
        FROM {} AS a, (
            SELECT ST_MAKEVALID(ST_SIMPLIFY(the_geom, {tolerance})) AS the_geom
            FROM {} WHERE cartodb_id = {}
        ) AS b
        WHERE st_intersects(a.the_geom, b.the_geom)
    """.format(wdpa_table, gadm_table2, cdbid, tolerance=tolerance)
    test_geoms.append( sendSql(query)['rows'] )

In [99]:
def areaEqual(geom):
    # Equal-area approx equiv to ST_AREA(geog) a la sgilles
    return shapely.ops.transform(
        partial(
            pyproj.transform,
            pyproj.Proj(init='EPSG:4326'),
            pyproj.Proj(
                proj='aea', # Albers equal area
                lat_1=geom.bounds[1],
                lat_2=geom.bounds[3])
        ), geom).area

def loadGeoms(rows, geom_field='the_geom'):
    return [shapely.wkb.loads(r[geom_field], hex=True) for r in rows]

In [100]:
## Using local postgis

%%timeit -r 1
for rows in test_geoms:
    geoms = ','.join(["'{}'::geometry".format(r['the_geom']) for r in rows])
    query = """
    SELECT ST_AREA(ST_UNION(ARRAY[{}])::geography) as area
    """.format(geoms)
    area = postgis(query)[0][0]
    print(area)

13256129663.4084
52754188196.3338
42641987.1496473
13256129663.4084
52754188196.3338
42641987.1496473
32.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [103]:
## Using shapely + pyproj

%%timeit -r 1
for rows in test_geoms:
    geoms = loadGeoms(rows)
    union_geoms = shapely.ops.cascaded_union(geoms)
    area = areaEqual(union_geoms)
    print(area)

13256076698.72476
52749935135.33423
42642012.0458095
13256076698.72476
52749935135.33423
42642012.0458095
1min 30s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
